In [16]:
import pandas as pd
import os

In [17]:
DATA_PATH = "../data/full"
leiden_metrics = pd.read_csv(os.path.join(DATA_PATH, "leiden", "metrics.csv"))
slpa_metrics = pd.read_csv(os.path.join(DATA_PATH, "w_slpa", "metrics.csv"))
reverse_hybrid_metrics = pd.read_csv(os.path.join(DATA_PATH, "reverse_hybrid", "metrics.csv"))

In [18]:
# Combine the three metrics DataFrames, add an 'algo' column, and concatenate with NaN for missing values
import numpy as np

# Helper to add algo column and standardize columns
def add_algo_col(df, algo_name):
    df = df.copy()
    df['algo'] = algo_name
    return df

# If leiden_metrics has a 'level' column, split by level
leiden_dfs = []
if 'level' in leiden_metrics.columns:
    for level in leiden_metrics['level'].unique():
        level_df = leiden_metrics[leiden_metrics['level'] == level].drop('level', axis=1)
        leiden_dfs.append(add_algo_col(level_df, f'leiden_level{level}'))
else:
    leiden_dfs.append(add_algo_col(leiden_metrics, 'leiden'))

slpa_df = add_algo_col(slpa_metrics, 'w_slpa')
reverse_hybrid_df = add_algo_col(reverse_hybrid_metrics, 'reverse_hybrid')

# Combine all DataFrames, aligning columns and filling missing with NaN
all_dfs = leiden_dfs + [slpa_df, reverse_hybrid_df]
combined_metrics = pd.concat(all_dfs, ignore_index=True, sort=True)

In [19]:
# transpose the DataFrame for better readability
combined_metric = combined_metrics.set_index('algo').T

row_order = [
    'modularity', 
    'overlapping_modularity', 
    'conductance', 
    'num_communities', 
    'avg_community_size', 
    'min_community_size', 
    'max_community_size', 
    'num_nodes_in_communities', 
    # 'nmi_epitype', 
    # 'nmi_subtype', 
    # 'ari_epitype', 
    # 'ari_subtype'
]

combined_metric.loc['overlapping_modularity', 'reverse_hybrid'] = np.nan
# Reorder the rows based on the specified order
combined_metrics  = combined_metric.loc[row_order]


In [20]:
combined_metrics

algo,leiden_level0,leiden_level1,leiden_level2,leiden_level3,w_slpa,reverse_hybrid
modularity,0.639343,0.430204,0.158592,-0.011439,0.612320,0.398796
overlapping_modularity,NaN,NaN,NaN,NaN,0.466719,NaN
conductance,0.011331,0.485594,0.677233,0.805265,0.164706,0.557392
num_communities,3.000000,8.000000,12.000000,2.000000,4.000000,10.000000
avg_community_size,336.000000,126.000000,73.000000,56.500000,265.500000,106.200000
min_community_size,286.000000,60.000000,47.000000,34.000000,66.000000,29.000000
max_community_size,372.000000,227.000000,114.000000,79.000000,379.000000,231.000000
num_nodes_in_communities,1008.000000,1008.000000,876.000000,113.000000,1008.000000,1008.000000


In [21]:
# for modularity, overlapping_modularity (non nan) and conductance use 3 decimal places
# for num_communities, min_community_size, max_community_size, num_nodes_in_communities use int, no decimal places
# for avg_community_size use 1 decimal places
def format_value(value):
    if pd.isna(value):
        return 'NaN'
    elif isinstance(value, float):
        if value.is_integer():
            return f"{int(value)}"
        elif value < 1:
            return f"{value:.3f}"
        else:
            return f"{value:.1f}"
    elif isinstance(value, int):
        return str(value)
    else:
        return str(value)
    

# Apply formatting to the DataFrame
formatted_metrics = combined_metrics.applymap(format_value)
formatted_metrics

/var/folders/_x/_k57kbwj3f11_drmx91npc7w0000gn/T/ipykernel_55223/2731292692.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_metrics = combined_metrics.applymap(format_value)


algo,leiden_level0,leiden_level1,leiden_level2,leiden_level3,w_slpa,reverse_hybrid
modularity,0.639,0.430,0.159,-0.011,0.612,0.399
overlapping_modularity,NaN,NaN,NaN,NaN,0.467,NaN
conductance,0.011,0.486,0.677,0.805,0.165,0.557
num_communities,3,8,12,2,4,10
avg_community_size,336,126,73,56.5,265.5,106.2
min_community_size,286,60,47,34,66,29
max_community_size,372,227,114,79,379,231
num_nodes_in_communities,1008,1008,876,113,1008,1008
